In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression,SGDClassifier
from sklearn.metrics import r2_score,accuracy_score,roc_auc_score
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        url = os.path.join(dirname, filename)
        print(url)
        df = pd.read_csv(url)
        
df.head()

# **Columns**    
**age**: Age of the patient (in years)  
**sex**: Sex of the patient (1 = male, 0 = female)  
**cp**: Chest pain type (1-4)  
**trestbps**: Resting blood pressure (in mm Hg on admission to the hospital)  
**chol**: Serum cholesterol in mg/dl  
**fbs**: Fasting blood sugar > 120 mg/dl (1 = true; 0 = false)  
**restecg**: Resting electrocardiographic results (0-2)  
**thalach**: Maximum heart rate achieved  
**exang**: Exercise-induced angina (1 = yes; 0 = no)  
**oldpeak**: ST depression induced by exercise relative to rest  
**slope**  
**ca**  
**thal**  
**target**: identifies whether given person has a heart disease (1 or 0)  

In [ ]:
df.info()


In [ ]:
navals = df.isna()
navals.any()
# There's no NA or NULL value in the dataset
# Totally there's 303 records in the dataset

In [ ]:
import warnings
warnings.filterwarnings("ignore", "use_inf_as_na")

# Distributions of features
pd.options.mode.chained_assignment = None
num_of_cols = len(df.columns)
fig,axes = plt.subplots(figsize=(12,24),ncols=2,nrows=7)
axes = axes.flatten()
for i,col in enumerate(df.columns):
    sns.histplot(data=df[col],ax=axes[i])
    axes[i].set_title(f'Distribution of {col}')
plt.subplots_adjust(hspace=0.5)


In [ ]:
print(df['age'].mode())
print(df['target'].value_counts(normalize=True))

* Distribution of **thalach** is left skewed. There will be a lot of data in the right tail of this feature
* Distribution of **cholesterol** appears to be normal excluding a couple of outliers (>=400)
* Distribution of **age** is slightly left skewed with mode at 58 years
* There are about **2 times more males than females** in this dataset
* There are about **2 times more people with exercise-induced angina** than those people who don't face this medical condition
* **Target value** (indicating heart disease) is roughly evenly distributed with 54%/46% split


In [ ]:
fig,axes = plt.subplots(figsize=(12,18),nrows=2,ncols=2)
axes = axes.flatten()
cols = ['thalach','chol','trestbps','oldpeak']
for i,col in enumerate(cols):
    sns.scatterplot(x=df['age'],y=df[col],hue=df['sex'],ax=axes[i])
    axes[i].set_title(col)

* Thalach which is **maximum heart rate** slightly correlates with age. This pattern is visible on the chart
* Similar with resting bps which follows a similar pattern but the other way around. The older the person gets, the higher resting bps gets
* Even though there's two time more males than females, one of the highest noted cholesterol levels belong to women

In [ ]:
fig,axes = plt.subplots(figsize=(12,6),nrows=1,ncols=2)

def get_r2(x,y):
    x = x.to_numpy().reshape(-1,1)
    lr = LinearRegression()
    lr.fit(x,y)
    y_pred = lr.predict(x)
    r2 = r2_score(y,y_pred)
    return r2

sns.regplot(x=df['age'],y=df['thalach'],ax=axes[0])
axes[0].set_title('Regression max heart rate')
thalachR2 = get_r2(df['age'],df['thalach'])

sns.regplot(x=df['age'],y=df['trestbps'],ax=axes[1])
axes[1].set_title('Regression rest bps')
trestbpsR2 = get_r2(df['age'],df['trestbps'])

print(thalachR2,trestbpsR2)

### One-hot encoding

In [ ]:
cols = ['cp','slope','ca','thal']
df = pd.get_dummies(df,columns=cols)
df

R2 score isn't really high but enough to discern a slight dependece on the chart

In [ ]:
plt.subplots(figsize=(18,14))
corr_data = df.corr()
sns.heatmap(corr_data,annot=True, cmap='Blues')

**Target** shows the strongest correlaton with given features:
* **-0.52** cp_0 (chest pain type) 
* **0.42** thalach (max heart rate achieved)
* **-0.44** exang (excercise-induced angina)
* **-0.43** oldpeak (ST depression induced by exercise relative to rest)
* **0.47** ca_0 
* **0.52** thal_2
* **-0.49** thal_3   

Cholesterol and fasting blood sugar having very low correlateion to target doesn't seem to be a good predictor whether a person has a heart disease or not

In [ ]:
cols_to_drop = np.array(corr_data[np.abs(corr_data['target'])<0.1]['target'].index)
print(cols_to_drop)

df.drop(columns=cols_to_drop,inplace=True)

In [ ]:
df.head()

### Data splitting



In [ ]:
X = df.drop(columns=['target'])
y = df['target']

X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.7,random_state=42,stratify=y,shuffle=True)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)
print(X_train.head())
print(y_train.head())

sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)


### Support Vector Machine

In [ ]:
clf = SVC(class_weight='balanced',random_state=42)
params = {
    'C':[0.01,0.1,1,10,100],
    'gamma':[1e-4,1e-5,1e-6],
    'kernel':['rbf','linear','poly']
}
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(clf,params,cv=5)
grid_search.fit(X_train_sc,y_train)
svc_best_params = grid_search.best_params_
print(f'Best parameters: {svc_best_params}')

In [ ]:
print('Best average accuracy out of 5 cross validation iterations')
grid_search.best_score_

In [ ]:
clf_best = grid_search.best_estimator_
y_pred = clf_best.predict(X_test_sc)
print(accuracy_score(y_test,y_pred))
wrong = [i for i in range(len(y_pred)) if y_pred[i]!=y_test.to_numpy()[i]]
wrong
print(clf_best.score(X_test_sc,y_test))
best_params = grid_search.best_params_

In [ ]:
random_forest = RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=-1,random_state=42)
random_forest.fit(X,y)
feat_importance = np.argsort(random_forest.feature_importances_)
random_forest.feature_importances_,feat_importance

### Feature selection

In [ ]:
num_of_feats = X.shape[1]
feats_arr = [i for i in range(1,int(num_of_feats+1))] # [1,3,...,num_of_feats]
feat_indices = [feat_importance[-i:] for i in feats_arr]
print(feat_indices)
svc_models = []
results = []
for feats in feat_indices:
    X_train_sel =  X_train_sc[:,feats]
    X_test_sel = X_test_sc[:,feats]

    svc_model = SVC(**best_params,random_state=42)
    svc_models.append(svc_model)
    score_k_feats = cross_val_score(svc_model,X_train_sel,y_train,).mean()
    results.append(score_k_feats)
    print(f' Models accuracy for {len(feats)} features is {score_k_feats}')

In [ ]:
plt.plot([i+1 for i in range(num_of_feats)],results)

The score plateus after 5th features and doesn't drop below threshold of 84%

I'm going to compare 2 set of features. 
* simpler with 6 features
* more complex with 16 features

In [ ]:
num_of_features = 6
X_train_new = X_train_sc[:,feat_indices[num_of_features-1]]
X_test_new = X_test_sc[:,feat_indices[num_of_features-1]]

svc_models[num_of_features-1].fit(X_train_new,y_train) 
print(f'Accuracy for {num_of_features} features:  {svc_models[num_of_features-1].score(X_test_new,y_test)}')

num_of_features = 18
X_train_new = X_train_sc[:,feat_indices[num_of_features-1]]
X_test_new = X_test_sc[:,feat_indices[num_of_features-1]]

svc_models[num_of_features-1].fit(X_train_new,y_train) 
svc_models[num_of_features-1].score(X_test_new,y_test)
print(f'Accuracy for {num_of_features} features:  {svc_models[num_of_features-1].score(X_test_new,y_test)}')

6 features turned out to be marginally better. For the rest of cross validation scores I'm going to use 17 features

In [ ]:
def grid_search(classifier,x_tr,y_tr,params):
    gridsearch = GridSearchCV(classifier,params)
    gridsearch.fit(x_tr,y_tr)
    best_params = gridsearch.best_params_
    return best_params
def grid_search_cv(classifier_name,x_tr,y_tr,best_params):
    classifier = classifier_name(**best_params)
    kvalscore = cross_val_score(classifier,x_tr,y_tr)
    return kvalscore.mean()

In [ ]:
sgd_lr = SGDClassifier(loss='log_loss',penalty=None,fit_intercept=True,max_iter=10000,learning_rate='adaptive',eta0=0.001,random_state=42)
sgd_lr.fit(X_train_new,y_train)
y_pred = sgd_lr.predict_proba(X_test_new)[:,1]
y_pred2 = sgd_lr.predict(X_test_new)
cross_val_sgd_lr = cross_val_score(sgd_lr,X_train_new,y_train).mean()
print(f'Area under ROC curve: {roc_auc_score(y_test,y_pred):.3f} ')
print(f'Cross val score: {cross_val_sgd_lr}')

In [ ]:
params = {
    'learning_rate': ['adaptive','constant','invscaling','optimal'],
   'eta0':[0.1,0.01,0.005,0.001,0.0001],
    'penalty': ['l1','l2'],
    'loss':['log_loss'],
    'max_iter':[1000,10000]
}
# sgd_grid_search = GridSearchCV(SGDClassifier(),params)
# sgd_grid_search.fit(X_train_new,y_train)
sgd_best_params = grid_search(SGDClassifier(random_state=42),X_train_new,y_train,params)
sgd_best_params

In [ ]:
sgd_lr = SGDClassifier(**sgd_best_params,random_state=42)
sgd_lr.fit(X_train_new,y_train)
cross_val_sgd_lr = cross_val_score(sgd_lr,X_train_new,y_train).mean()
print(f'Cross val score: {cross_val_sgd_lr}')


The best I could achieve with logistic regression is **~87%**

### K-neighbours algorithm


In [ ]:
knn = KNeighborsClassifier(algorithm='auto')
params = {
    'n_neighbors':[3,5,7,9,11,13,15],
    'weights' :['uniform', 'distance'],
    'metric': ['minkowski', 'chebyshev','manhattan'],
    'p':[2],
    'weights': ['uniform', 'distance'],
}
knn_best_params = grid_search(knn,X_train_new,y_train,params)
print(knn_best_params)

print(f'Cross val score for knn classifier: {grid_search_cv(KNeighborsClassifier,X_train_new,y_train,knn_best_params)}')
# knn_model = KNeighborsClassifier(**knn_best)
# knn_model.fit(X_train_new,y_train)

**K-neighbours algorithm** allowed me to increase accuracy by additional 1% -> **~86%**

Now let's check out how Random forest classifier will perform

In [ ]:
rfc = RandomForestClassifier(n_estimators=100,max_depth=5,min_samples_leaf=2,min_samples_split=4,n_jobs=-1)
print(cross_val_score(rfc,X_train_new,y_train).mean())

params = {
    'max_depth': [3,5,7],
#     'min_samples_split': [1,2,3],
#     'criterion': ['gini','log_loss','entropy'],
#     'min_samples_leaf': [1,2,3],
    'n_estimators': [50,100,200,500,1000]
}
rfc_best_params = grid_search(RandomForestClassifier(n_jobs=-1,random_state=42),X_train_new,y_train,params)
print(rfc_best_params)

In [ ]:
rfc_best = grid_search_cv(RandomForestClassifier,X_train_new,y_train,rfc_best_params)
print(f'Cross val score for Random Forest classifier: {rfc_best}')

**Random Forest Classifier** achieved similar result as K-neighbours algorithm **~86%**

And now it's time for the last but not least XGB Boost Classifier

In [ ]:
xgb = XGBClassifier(n_estimators=500,learning_rate=0.01,max_depth=3)
cross_val_score(xgb,X_train_new,y_train,n_jobs=-1).mean()

params = {
    'max_depth': [3,4,5,6,7],
    'n_estimators': [500,1000,1500],
    'learning_rate':[0.0005,0.001,0.01,0.02]
}

xgb_best_params = grid_search(XGBClassifier(random_state=42),X_train_new,y_train,params)
print(xgb_best_params)

In [ ]:
xgb_best = grid_search_cv(XGBClassifier,X_train_new,y_train,xgb_best_params)
print(f'Cross val score for XGB Classifier: {xgb_best}')

In [ ]:

models = [XGBClassifier(**xgb_best_params),RandomForestClassifier(**rfc_best_params),KNeighborsClassifier(**knn_best_params),SGDClassifier(**sgd_best_params),
         SVC(**svc_best_params)]
names=['XGB','RF','KN','SGD','SVC']

X_train_6feats = X_train_sc[:,feat_indices[5]]
X_test_6feats = X_test_sc[:,feat_indices[5]]
X_train_18feats = X_train_sc[:,feat_indices[17]]
X_test_18feats = X_test_sc[:,feat_indices[17]]
scores_6 = []
scores_18 = []

for model in models:
    model.fit(X_train_6feats,y_train)
    scores_6.append(model.score(X_test_6feats,y_test))
    
    model.fit(X_train_18feats,y_train)
    scores_18.append(model.score(X_test_18feats,y_test))
    print(f'model {model} done')


In [ ]:
data = {
    'Name': names * 2,
    'Score': scores_6 + scores_18,  
    'Type': ['Scores_6'] * len(scores_6) + ['Scores_18'] * len(scores_18)
}
print(data)
axes = sns.barplot(x='Name', y='Score', hue='Type', data=data)
plt.ylim(0.7)

for bar in axes.containers:
    axes.bar_label(bar, fmt='%.2f')


In **4 cases** more **complex model** had better accuracy (XGB,RF,KN,SGD), while for the last one (SVC), the simpler model performed slightly better

After experimenting with various ML algorithms, the best result can be attributed to **KNeighbors Classifier** which achieved **~85%** on a test set